# Sparse Hebbian Learning : testing different rescaling functions

In this notebook, we test the convergence of SparseNet as a function of different parameters tuning the quantization. These parameters only influence 



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [ ]:
from shl_scripts.shl_experiments import SHL

list_figures = ['show_dico']#, 'plot_variance',  'plot_variance_histogram',  'time_plot_prob',  'time_plot_kurt',  'time_plot_var']
DEBUG_DOWNSCALE, verbose = 10, 0
DEBUG_DOWNSCALE, verbose = 10, 100
DEBUG_DOWNSCALE, verbose = 1, 0
N_scan = 7
n_jobs = 7
opts = dict(DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, alpha_homeo=0.,
              learning_algorithm='mp', verbose=verbose)
shl = SHL(**opts)
data = shl.get_data()

We will use the ``joblib`` package do distribute this computation on different CPUs.

In [ ]:
from joblib import Parallel, delayed
Parallel(n_jobs=n_jobs)(delayed(np.sqrt)(i ** 2) for i in range(10))

## different rescaling values

In [ ]:
!ls -l data_cache

In [ ]:
!ls -l data_cache

In [ ]:
from joblib import Parallel, delayed

def run(C, list_figures, data):
    matname = 'rescaling - C={}'.format(C)
    shl = SHL(DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, 
              C=C, verbose=verbose)
    dico = shl.learn_dico(data=data, matname=matname, list_figures=list_figures)
    return dico


Cs = np.linspace(0, 10, 5)
if not n_jobs==1: out = Parallel(n_jobs=n_jobs, verbose=15)(delayed(run)(C, [], data) for C in Cs)

for C in Cs:
    dico = run(C, list_figures=list_figures, data=data)
    plt.show()

## different quantization parameters

In [ ]:
2 ** np.arange(3, 9)

with a fixed $C=5.$

In [ ]:
from joblib import Parallel, delayed
def run(nb_quant, list_figures, data):
    matname = 'rescaling - nb_quant={}'.format(nb_quant)
    shl = SHL(DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, 
              C=5., nb_quant=nb_quant, verbose=verbose)
    dico = shl.learn_dico(data=data, matname=matname, list_figures=list_figures)
    return dico

nb_quants = 2 ** np.arange(3, 9)
if not n_jobs==1: out = Parallel(n_jobs=n_jobs, verbose=15)(delayed(run)(int(nb_quant), [], data) for nb_quant in  nb_quants)

for nb_quant in  nb_quants:
    dico = run(nb_quant, list_figures, data)
    plt.show()

with an adaptive rescaling function ($C=0.$)

In [ ]:
from joblib import Parallel, delayed
def run(nb_quant, list_figures, data):
    matname = 'rescaling - auto - nb_quant={}'.format(nb_quant)
    shl = SHL(DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, 
              C=0., nb_quant=nb_quant, verbose=verbose)
    dico = shl.learn_dico(data=data, matname=matname, list_figures=list_figures)
    return dico

nb_quants = 2 ** np.arange(3, 9)
if not n_jobs==1: out = Parallel(n_jobs=n_jobs, verbose=15)(delayed(run)(int(nb_quant), [], data) for nb_quant in  nb_quants)

for nb_quant in  nb_quants:
    dico = run(nb_quant, list_figures, data)
    plt.show()

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts